In [1]:
# https://medium.com/@lilianli1922/authenticating-vertex-ai-gemini-api-calls-in-python-using-service-accounts-without-gcloud-cli-e17203995ff1

In [2]:
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.oauth2 import service_account
import json
import os

def load_credentials_from_file(service_account_path: str):
    """Loads Google Cloud credentials from a service account JSON file."""
    try:
        return service_account.Credentials.from_service_account_file(service_account_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"Service account file not found: {service_account_path}")
    except Exception as e:
        raise RuntimeError(f"Error loading credentials from {service_account_path}: {e}")

def get_project_id_from_file(service_account_path: str) -> str:
    """Extracts the project_id from the service account JSON file."""
    try:
        with open(service_account_path, 'r') as f:
            data = json.load(f)
            project_id = data.get("project_id")
            if not project_id:
                raise ValueError("Key 'project_id' not found in service account file.")
            return project_id
    except FileNotFoundError:
        raise FileNotFoundError(f"Service account file not found: {service_account_path}")
    except (json.JSONDecodeError, ValueError, KeyError) as e:
        raise RuntimeError(f"Error reading project_id from {service_account_path}: {e}")

def generate_text_vertexai(
    project_id: str,
    location: str,
    credentials, # Pass the loaded credentials object
    prompt: str,
    model_name: str = 'gemini-2.0-flash', # Default model, can be overridden
    temperature: float = 0.9, # Example generation parameter
    max_output_tokens: int = 256 # Example generation parameter
) -> str:
    """
    Generates text using a Gemini model via the Vertex AI API with explicit credentials.

    Args:
        project_id: The Google Cloud project ID.
        location: The Google Cloud region (e.g., "us-central1").
        credentials: The loaded google.oauth2.service_account.Credentials object.
        prompt: The text prompt for the model.
        model_name: The name of the Gemini model to use (e.g., "gemini-1.5-flash").
        temperature: Controls randomness (0.0-1.0). Higher values are more creative.
        max_output_tokens: Maximum number of tokens in the generated response.

    Returns:
        The generated text from the model. Returns an empty string on errors
        where no candidate is available. Raises exceptions for setup/auth errors.
    """
    print(f"Initializing Vertex AI for project: {project_id}, location: {location}")
    try:
        vertexai.init(project=project_id, location=location, credentials=credentials)

        print(f"Loading model: {model_name}")
        model = GenerativeModel(model_name)

        generation_config = GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens
        )

        print(f"Sending prompt: '{prompt[:50]}...'") # Show beginning of prompt
        response = model.generate_content(
            prompt,
            generation_config=generation_config
        )

        print("Response received.")
        # Basic check: Ensure candidates list is not empty and has content
        if response.candidates and response.candidates[0].content.parts:
            return response.candidates[0].content.parts[0].text
        else:
            # Log or handle cases with no valid response (e.g., safety filters)
            print(f"Warning: No valid response candidates found. Response: {response}")

            return ""

    except Exception as e:
        print(f"An error occurred during Vertex AI text generation: {e}")
        # Depending on requirements, you might re-raise, return None, or return ""
        return ""


In [ ]:
from configparser import ConfigParser
config = ConfigParser()

config.read('.keys/config.ini')
PROJECT_ID = config.get('google', 'project_id')
SERVICE_ACCOUNT_FILE = config.get('google', 'service_account_file')
LOCATION = config.get('google', 'location')
MODEL_NAME = config.get('google', 'default_model')

In [4]:
USER_PROMPT = "Who is Albert Laszlo Barabasi?"

In [5]:

# --- Authentication and Setup ---
if not os.path.exists(SERVICE_ACCOUNT_FILE):
        raise FileNotFoundError(f"Critical: Service account file not found at {SERVICE_ACCOUNT_FILE}")

# Load credentials object
creds = load_credentials_from_file(SERVICE_ACCOUNT_FILE)

# Extract project ID (can also be passed directly if known)
proj_id = get_project_id_from_file(SERVICE_ACCOUNT_FILE)

# --- Generate Text ---
generated_text = generate_text_vertexai(
    project_id=proj_id,
    location=LOCATION,
    credentials=creds,
    prompt=USER_PROMPT,
    model_name=MODEL_NAME
)

# --- Output ---
if generated_text:
    print("\n--- Generated Text ---")
    print(generated_text)
else:
    print("\nFailed to generate text or received an empty response.")

Initializing Vertex AI for project: fengroland, location: us-central1
Loading model: gemini-2.0-flash
Sending prompt: 'Who is Albert Laszlo Barabasi?...'


/Users/espinl/anaconda3/envs/py311LLMEval/lib/python3.11/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Response received.

--- Generated Text ---
Albert-László Barabási is a prominent physicist and network scientist, best known for his pioneering work in the field of network science, specifically for his discovery of scale-free networks. Here's a more detailed breakdown:

*   **Professional Background:** Barabási is a Hungarian-born Romanian-American physicist. He holds the position of Robert Gray Dodge Professor of Network Science and Distinguished University Professor at Northeastern University, where he directs the Center for Complex Network Research. He also holds appointments at the Central European University in Budapest and is a member of the Hungarian Academy of Sciences.

*   **Key Contributions:** His most significant contribution is the discovery of **scale-free networks**.  These networks are characterized by a power-law degree distribution, meaning that a small number of nodes (hubs) have a very high number of connections, while most nodes have only a few connections.  This